# 🎬 ViralCutter - Smooth Zoom Edition
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/masterface77/ViralCutter/blob/smooth-zoom/ViralCutter-SmoothZoom.ipynb)

Versão especial com **YOLO Tracking + Smooth Zoom** automático!

- ✅ Download de vídeos do YouTube
- ✅ Cortes automáticos com IA (Gemini)
- ✅ Legendas dinâmicas
- ✅ **NOVO**: Câmera suave que segue o rosto (EMA Smoothing)

---
[![Discord](https://dcbadge.limes.pink/api/server/tAdPHFAbud)](https://discord.gg/tAdPHFAbud)

In [ ]:
#@title 🚀 INICIAR VIRALCUTTER (SMOOTH ZOOM)
#@markdown Execute esta célula para instalar e iniciar a interface.

import os
import shutil
import subprocess
import threading
import time
import sys
from IPython.display import clear_output
from google.colab import drive

# ==========================================
# ⚙️ CONFIGURAÇÕES
# ==========================================
PASTA_DRIVE_FINAL = "/content/drive/MyDrive/ViralCutter_SmoothZoom"
REPO_URL = "https://github.com/masterface77/ViralCutter.git"
BRANCH = "smooth-zoom"  # Branch com Smooth Zoom

# ==========================================
# 1. MONTAGEM DO DRIVE
# ==========================================
print("1️⃣ Conectando ao Google Drive...")
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

os.makedirs(PASTA_DRIVE_FINAL, exist_ok=True)
print(f"✅ Drive conectado! Vídeos irão para: {PASTA_DRIVE_FINAL}")

# ==========================================
# 2. INSTALAÇÃO (BRANCH SMOOTH-ZOOM)
# ==========================================
if not os.path.exists("/content/ViralCutter"):
    print(f"\n📦 Clonando ViralCutter (branch: {BRANCH})...")

    # Clone da branch smooth-zoom diretamente
    subprocess.run(f"git clone -b {BRANCH} {REPO_URL} /content/ViralCutter", shell=True)
    os.chdir("/content/ViralCutter")

    # UV (gerenciador rápido)
    subprocess.run("pip install uv -q", shell=True)

    # Drivers de Sistema
    subprocess.run("apt-get update -y -qq && apt-get install -y libcudnn8 ffmpeg xvfb -qq", shell=True)

    # Cria ambiente virtual
    subprocess.run(["uv", "venv", ".venv"], check=True)

    # Instalação das dependências
    print("   🚀 Instalando dependências + Ultralytics...")
    cmds = [
        "uv pip install --python .venv faster-whisper",
        "uv pip install --python .venv git+https://github.com/m-bain/whisperx.git",
        "uv pip install --python .venv -r requirements.txt",
        "uv pip install --python .venv -U --pre 'yt-dlp[default]'",
        "uv pip install --python .venv google-generativeai pandas onnxruntime-gpu",
        "uv pip install --python .venv transformers==4.46.3 accelerate>=0.26.0",
        "uv pip install --python .venv torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121",
        "uv pip install --python .venv insightface",
        # YOLO Smooth Zoom
        "uv pip install --python .venv ultralytics",
        "uv pip uninstall --python .venv mediapipe protobuf flatbuffers",
        "uv pip install --python .venv 'mediapipe>=0.10.0' 'protobuf>=3.20,<5.0' 'flatbuffers>=2.0'"
    ]

    for cmd in cmds:
        subprocess.run(cmd, shell=True, check=True)

    print("✅ Instalação Concluída!")
else:
    os.chdir("/content/ViralCutter")
    print("\n✅ Sistema já instalado.")

# ==========================================
# 3. MONITOR DE ARQUIVOS (DRIVE SYNC)
# ==========================================
def engine_loop():
    print("👀 Engine: Monitorando pasta VIRALS...")
    processed = set()
    WATCH_DIR = "/content/ViralCutter/VIRALS"

    while True:
        if os.path.exists(WATCH_DIR):
            for r, d, f in os.walk(WATCH_DIR):
                for file in f:
                    if not file.endswith(".mp4"): continue
                    if any(x in file for x in ["input", "temp", "original"]): continue

                    full_path = os.path.join(r, file)

                    if full_path not in processed and os.path.getsize(full_path) > 1e6:
                        s1 = os.path.getsize(full_path)
                        time.sleep(5)
                        if os.path.getsize(full_path) != s1: continue

                        print(f"\n💎 NOVO VÍDEO: {file}")
                        print(f"   ☁️ Enviando para Drive...")
                        shutil.copy(full_path, os.path.join(PASTA_DRIVE_FINAL, os.path.basename(full_path)))
                        print("   ✅ Salvo no Drive!")

                        processed.add(full_path)

        time.sleep(5)

threading.Thread(target=engine_loop, daemon=True).start()

# ==========================================
# 4. INICIAR INTERFACE GRADIO
# ==========================================
print("\n🚀 INICIANDO INTERFACE SMOOTH ZOOM...")
print("⚠️ CLIQUE NO LINK gradio.live ABAIXO:")
print("="*60)

# Display Virtual
subprocess.Popen(['Xvfb', ':1', '-screen', '0', '2560x1440x8'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(2)
os.environ.update({'DISPLAY':':1.0', 'MPLBACKEND':'Agg', 'CUDA_VISIBLE_DEVICES':'0', 'VIRALCUTTER_FACE_MODEL': 'yolo'})

# Roda o App com YOLO como padrão
!/content/ViralCutter/.venv/bin/python webui/app.py --colab --face-model yolo

## 📝 Créditos

**ViralCutter** por [Rafa.Godoy](https://github.com/rafaelGodoyEbert)

**Smooth Zoom** implementado com:
- [Ultralytics YOLOv8](https://github.com/ultralytics/ultralytics)
- ByteTrack para tracking persistente
- EMA (Exponential Moving Average) para suavização

---
`v0.8 Alpha + Smooth Zoom`